In [1]:
import json

import datetime

from influxdb_client import InfluxDBClient, Point, Dialect
from influxdb_client.client.write_api import SYNCHRONOUS

In [2]:
host = "http://localhost:8086"
token = "mB_Kr-Ak8SwqPByhbH25ozucJGE5DUAzOF1zLRLoJ39ndpBa-0NAaE6FWp7AHeY6323P2Gv2ESi28IycphHXZg=="
org = "iciclespy3"
bucket = "iciclespy3"

In [3]:
client = InfluxDBClient(url=host, token=token, org=org)
query_api = client.query_api()

In [15]:
datetime.datetime.strptime('18/09/2029 01:55:19', "%d/%m/%Y %H:%M:%S")

datetime.datetime(2029, 9, 18, 1, 55, 19)

In [4]:
start = '18/09/2019 01:55:19'
end = '18/09/2021 01:55:19'

p = {"_start": datetime.datetime.strptime(start, "%d/%m/%Y %H:%M:%S"),
     '_end': datetime.datetime.strptime(end, "%d/%m/%Y %H:%M:%S"),
     '_temperature': 'temperature'}

tables = query_api.query(f'from(bucket:"{bucket}") |> range(start: _start, stop: _end) |> filter(fn: (r) => r._measurement == _temperature)', params=p)

# p = {"_start": datetime.datetime.strptime(start, "%d/%m/%Y %H:%M:%S"),
#      '_end': datetime.datetime.strptime(end, "%d/%m/%Y %H:%M:%S"),
#      '_my_measurement': 'temperature',
#      '_min_t': -20,
#      '_max_t': 20}

# tables = query_api.query(f'from(bucket:"{bucket}") |> range(start: _start, stop: _end) |> filter(fn: (r) => r._measurement == _my_measurement and _min_t < r._value and r._value < _max_t)', params=p)

In [5]:
time_series = {}
time_series['count'] = len(tables[0].records)
time_series['records'] = []

In [6]:
for table in tables:
    for record in table.records:
        tmp = {}
        tmp['time'] = record['_time'].strftime("%d/%m/%Y %H:%M:%S")
        
        tmp['country'] = record['country']
        tmp['region_state'] = record['region_state']
        tmp['city'] = record['city']
        tmp['street'] = record['street']
        tmp['building'] = record['building']
        tmp['index'] = record['index']
                
        tmp['measurement'] = record['_measurement']
        tmp['value'] = round(record['_value'], 3)
        
        time_series['records'].append(tmp)

In [7]:
time_series

{'count': 3,
 'records': [{'time': '05/05/2021 12:56:20',
   'country': 'Russia',
   'region_state': 'Moscow',
   'city': 'Moscow',
   'street': 'Taganskaya',
   'building': '25-27',
   'index': '1',
   'measurement': 'temperature',
   'value': 11.505},
  {'time': '05/05/2021 12:56:36',
   'country': 'Russia',
   'region_state': 'Moscow',
   'city': 'Moscow',
   'street': 'Taganskaya',
   'building': '25-27',
   'index': '1',
   'measurement': 'temperature',
   'value': 15.295},
  {'time': '05/05/2021 12:56:55',
   'country': 'Russia',
   'region_state': 'Moscow',
   'city': 'Moscow',
   'street': 'Taganskaya',
   'building': '25-27',
   'index': '1',
   'measurement': 'temperature',
   'value': -3.551}]}

In [8]:
time_series = json.dumps(time_series)
time_series

'{"count": 3, "records": [{"time": "05/05/2021 12:56:20", "country": "Russia", "region_state": "Moscow", "city": "Moscow", "street": "Taganskaya", "building": "25-27", "index": "1", "measurement": "temperature", "value": 11.505}, {"time": "05/05/2021 12:56:36", "country": "Russia", "region_state": "Moscow", "city": "Moscow", "street": "Taganskaya", "building": "25-27", "index": "1", "measurement": "temperature", "value": 15.295}, {"time": "05/05/2021 12:56:55", "country": "Russia", "region_state": "Moscow", "city": "Moscow", "street": "Taganskaya", "building": "25-27", "index": "1", "measurement": "temperature", "value": -3.551}]}'